# Resources

https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/

https://github.com/vwxyzjn/cleanrl?tab=readme-ov-file

In [1]:
import gymnasium as gym
import seaborn as sns
import os
from collections import deque, Counter, namedtuple, defaultdict
import random
from matplotlib import pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import torch
from torch import nn
from torch.nn import init
import torch.nn.functional as F
from torch.distributions import Categorical
import math
from itertools import count
from tqdm import tqdm
import numpy as np
import time
import uuid

from stable_baselines3.common.atari_wrappers import ClipRewardEnv, FireResetEnv, MaxAndSkipEnv, NoopResetEnv

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Params

In [8]:
ENV_ARGS = {
    'id': "BreakoutNoFrameskip-v4"
}
NUM_ENVS = 3
SEED = 1
LR = 3e-4
NUM_STEPS = 2048
NUM_ITERATIONS = 1000
GAMMA = 0.99
GAE_LAMBDA = 0.95
UPDATE_EPOCHS = 10
CLIP_COEF = 0.2 # the epsilon in KL divergece in PPO paper
ENTROPY_COEF = 0.0
VF_COEF = 0.5
MAX_GRAD_NORM = 0.5
MINI_BATCH_COUNT = 32
UPDATE_PLOTS = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device = ', DEVICE)

#output directory
ROOT = os.getcwd()
OUTPUT = os.path.join(ROOT, 'output')

if os.path.exists(OUTPUT) == False:
    os.makedirs(OUTPUT)

#seeding
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device =  cuda


In [9]:
gym.envs.registration.registry.keys()

dict_keys(['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'CartPoleJax-v0', 'CartPoleJax-v1', 'PendulumJax-v0', 'LunarLander-v2', 'LunarLanderContinuous-v2', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v2', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'Jax-Blackjack-v0', 'Reacher-v2', 'Reacher-v4', 'Pusher-v2', 'Pusher-v4', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4', 'Swimmer-v2', 'Swimmer-v3', 'Swimmer-v4', 'Walker2d-v2', 'Walker2d-v3', 'Walker2d-v4', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Humanoid-v2', 'Humanoid-v3', 'Humanoid-v4', 'HumanoidStandup-v2', 'HumanoidStandup-v4', 'GymV26Environment-v0', 'GymV21Environment-v0', 'Adventure-v0', 'AdventureDeterministic-v0', 'AdventureNoFrameskip-v0', 'Adventure-v4', 'AdventureDeterministic-

In [10]:
env = gym.make(**ENV_ARGS)

# Make envs

In [17]:
def make_env(**env_args):
    env = gym.make(**env_args)
    # env = gym.wrappers.FlattenObservation(env)
    env = gym.wrappers.RecordEpisodeStatistics(env)
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip = 4)

    env = ClipRewardEnv(env)
    env = gym.wrappers.ResizeObservation(env, (84,84)) 
    env = gym.wrappers.GrayScaleObservation(env) 
    env = gym.wrappers.FrameStack(env, 4) 
    return env


In [20]:
# Test env
envs = gym.vector.SyncVectorEnv(
    [lambda : make_env(**ENV_ARGS) for _ in range(NUM_ENVS)]
) 

assert isinstance(envs.single_action_space, gym.spaces.Discrete), 'Only discrete action is supported'

# Model

In [51]:
def layer_init(layer: nn.Linear, std = np.sqrt(2), bias_const = 0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

class Agent(nn.Module):

    def __init__(self, envs: gym.Env, hidden_size: int = 512):

        super().__init__()

        self.network = nn.Sequential(
            layer_init(nn.Conv2d(4, 32, 8, stride = 4)),
            nn.ReLU(),
            layer_init(nn.Conv2d(32, 64, 4, stride = 2)),
            nn.ReLU(),
            layer_init(nn.Conv2d(64, 64, 3, stride = 1)),
            nn.ReLU(),
            nn.Flatten(),
            layer_init(nn.Linear(64 * 7 * 7, hidden_size)),
            nn.ReLU(),
        )

        self.actor = layer_init(nn.Linear(hidden_size, envs.single_action_space.n), std = 0.01)
        self.critic = layer_init(nn.Linear(hidden_size,1 ), std = 1.0)
    
    def get_value(self, x):
        return self.critic(self.network(x/255.0))
    
    def get_action_and_value(self, x, action = None):
        '''
        @params:
            x: torch.tensor observation, shape = (N, observation size)
            action: torch.tensor action
        @returns:
            action: torch.tensor, shape = (N, action size)
            log_prob: torch.tensor, shape = (N,)
            entropy: torch.tensor, shape = (N,)
            value: torch.tensor, shape = (N,)
        '''

        hidden = self.network(x/255.0)
        logits = self.actor(hidden)
        probs = Categorical(logits=logits)
        if action == None:
            action = probs.sample()

        log_prob = probs.log_prob(action)
        entropy = probs.entropy()
        value = self.critic(hidden)
        return action, log_prob, entropy, value
  

In [53]:
#Test agent
# Test env
envs = gym.vector.SyncVectorEnv(
    [lambda : make_env(**ENV_ARGS) for _ in range(NUM_ENVS)]
) 

assert isinstance(envs.single_action_space, gym.spaces.Discrete), 'Only discrete action is supported'

obs, info = envs.reset()
obs = torch.tensor(obs).float()
print('obs shape = ', obs.shape)

test_agent = Agent(envs)

action, log_prob, entropy, value = test_agent.get_action_and_value(obs)

print('action shape = ', action.shape)
print('log prob shape = ', log_prob.shape)
print('entropy shape = ', entropy.shape)
print('value shape = ', value.shape)

envs.close()
del test_agent


obs shape =  torch.Size([3, 4, 84, 84])
action shape =  torch.Size([3])
log prob shape =  torch.Size([3])
entropy shape =  torch.Size([3])
value shape =  torch.Size([3, 1])


# Utils

In [54]:
def plot(history, show = False, save_path = None):
    sns.lineplot(y = history['reward'], x = list(range(len(history['reward']))))

    if save_path != None:
        plt.savefig(save_path)
    if show:
        plt.show()
        
    plt.clf()
    plt.close()

In [8]:
def evaluate(agent, episodes = 10):
    envs = gym.vector.SyncVectorEnv([lambda: make_env(gamma = GAMMA, **ENV_ARGS)])
    agent.eval()
    total_rewards = []
    next_obs, _ = envs.reset()

    while len(total_rewards) < episodes: 
        next_obs = torch.Tensor(next_obs)
        with torch.no_grad():
            action, log_prob, _, value = agent.get_action_and_value(next_obs)

        next_obs, reward, terminated, truncated, info = envs.step(action.numpy())

        if 'final_info' in info:
            for data in info['final_info']:
                if data:
                    reward = data['episode']['r'][0]
                    total_rewards.append(reward)

    return total_rewards

# Training loop

In [66]:
# Create env
envs = gym.vector.AsyncVectorEnv(
    [lambda : make_env(**ENV_ARGS) for _ in range(NUM_ENVS)]
) 

agent = Agent(envs).to(DEVICE)
optimizer = torch.optim.Adam(agent.parameters(), lr = LR, eps = 1e-5)

M = NUM_STEPS
N = NUM_ENVS

label = str(uuid.uuid4()).split('-')[0]
print('run id = ', label)

SAVE_PATH = os.path.join(OUTPUT, label)
FIG_SAVE_PATH = os.path.join(SAVE_PATH, 'plot.png')
if os.path.exists(SAVE_PATH) == False:
    print(f'output folder: {SAVE_PATH}')
    os.makedirs(SAVE_PATH)

obs = torch.zeros((M, N) + envs.single_observation_space.shape).to(DEVICE)
actions = torch.zeros((M,N) + envs.single_action_space.shape).to(DEVICE)
log_probs = torch.zeros((M,N)).to(DEVICE)
rewards = torch.zeros((M,N)).to(DEVICE)
dones = torch.zeros((M,N)).to(DEVICE) # for masking
values = torch.zeros((M,N)).to(DEVICE)

global_step = 0

next_obs, _ = envs.reset()
next_obs = torch.Tensor(next_obs).to(DEVICE)
next_done = torch.zeros(N).to(DEVICE) #N is num envs

print('next obs = ', next_obs.shape)
print('next done = ', next_done.shape)

reward_window = deque(maxlen = 100)
history = defaultdict(list)

loop = tqdm(range(NUM_ITERATIONS))
agent.train()

best_score = -float('inf')
evaluation = 0
loss = float('inf')

for iter in loop:

    #ROLLOUT phase
    #M is max steps
    if iter % UPDATE_PLOTS == 0:
        plot(history, save_path=FIG_SAVE_PATH)

    for step in range(M):
        global_step += N

        obs[step] = next_obs
        dones[step] = next_done

        #get action
        #NOTE: no_grad disables gradient calculation --> reduce memory consumption
        #the result of every computation will have requires_grad=False
        with torch.no_grad():
            action, log_prob, _, value = agent.get_action_and_value(next_obs)
            values[step] = value.flatten()

        actions[step] = action
        log_probs[step] = log_prob

        #make next step with actions
        next_obs, reward, terminated, truncated, info = envs.step(action.cpu().numpy())

        next_done = np.logical_or(terminated, truncated)

        #NOTE: difference between view and reshape
        # https://stackoverflow.com/questions/49643225/whats-the-difference-between-reshape-and-view-in-pytorch
        rewards[step] = torch.tensor(reward).view(-1)
        next_obs = torch.tensor(next_obs).to(DEVICE)
        next_done = torch.tensor(next_done).to(DEVICE)

        #NOTE: vector envs will automatically reset, so no need to break 
        if 'final_info' in info:
            for data in info['final_info']:
                if data:
                    reward = data['episode']['r']
                    reward_window.append(reward)
                    avg_reward = np.mean(reward_window)
                    history['reward'].append(avg_reward)
                    loop.set_description(f"reward = {avg_reward:.2f}, global_step = {global_step}, best_score = {best_score:.2f}, loss={loss:.2f}")

                    if best_score < avg_reward:
                        best_score = avg_reward
                        #save model
                        torch.save(agent, os.path.join(SAVE_PATH, 'ppo.checkpoint.torch'))
        
    #update the history for plotting, and printing progress

    #OPTIMIZE phase:
    with torch.no_grad():
        #bootstrap values, compute returns
        next_value = agent.get_value(next_obs).reshape(1,-1)
        advantages = torch.zeros_like(rewards).to(DEVICE)
        last_gae_lambda = 0

        for t in reversed(range(NUM_STEPS)):
            if t == NUM_STEPS - 1:
                next_none_terminal = np.logical_not(next_done.cpu())
                next_values = next_value
            else:
                next_none_terminal = np.logical_not(dones[t + 1].cpu())
                next_values = values[t + 1]
            
            #A(s,a) = Q(s,a) - V(s,a) = r(t) + gamma * V(s', a) * mask - V(s)
            # print(rewards[t].device)
            # print(values[t].device)
            # print(next_values.device)
            # print(next_none_terminal.device)
            next_none_terminal = next_none_terminal.to(DEVICE)

            delta = rewards[t] + GAMMA * next_values * next_none_terminal - values[t]
            #NOTE: learn about this formula
            advantages[t] = last_gae_lambda = delta + GAMMA * GAE_LAMBDA * next_none_terminal * last_gae_lambda
        returns = advantages + values
    
    #flatten the batch
    b_obs = obs.reshape((-1,) + envs.single_observation_space.shape)
    b_actions = actions.reshape((-1,) + envs.single_action_space.shape)
    b_log_probs = log_probs.reshape(-1)
    b_advantages = advantages.reshape(-1)
    b_returns = returns.reshape(-1)
    b_values = values.reshape(-1)

    #NOTE: randomize the batch to break correlation
    batch_size = M * N
    mini_batch_size = batch_size // MINI_BATCH_COUNT
    b_indicies = np.arange(batch_size)
    clip_fracs = []
    
    for epoch in range(UPDATE_EPOCHS):
        np.random.shuffle(b_indicies)

        #NOTE: mini-batch update: 
        # pros: reduce memory usage, faster updates
        # pros: a whole batch may stuck in local minima, mini batches introduce randomness
        # cons: estimate a true gradient, larger mini batch size --> more accurate but more memory
        for start in range(0, batch_size, mini_batch_size):
            end = start + mini_batch_size
            mini_indicies = b_indicies[start:end]

            _, new_log_prob, entropy, new_value = agent.get_action_and_value(b_obs[mini_indicies], b_actions[mini_indicies])

            #NOTE: what formula is this? 
            log_ratio = new_log_prob - b_log_probs[mini_indicies]

            ratio = log_ratio.exp() # trick to remove log

            #compute approximate KL: http://joschu.net/blog/kl-approx.html
            with torch.no_grad():
                old_approx_kd = (-log_ratio).mean()
                approximate_kl = ((ratio - 1) - log_ratio).mean()
                clip_fracs += [((ratio - 1.0).abs() > CLIP_COEF).float().mean().item()]

            mb_advantages = b_advantages[mini_indicies]

            #normalize advantage
            mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

            #policy loss (actor)

            pg_loss1 = -mb_advantages * ratio
            pg_loss2= -mb_advantages * torch.clamp(ratio, 1 - CLIP_COEF, 1 + CLIP_COEF)

            pg_loss = torch.max(pg_loss1, pg_loss2).mean()

            new_value = new_value.view(-1)

            #value loss (MSE)
            v_loss = 0.5 * ((new_value - b_returns[mini_indicies]) ** 2).mean()

            entropy_loss = entropy.mean()

            loss = pg_loss - ENTROPY_COEF * entropy_loss + v_loss * VF_COEF

            optimizer.zero_grad()
            loss.backward()
            #clip grad
            nn.utils.clip_grad_norm_(agent.parameters(), MAX_GRAD_NORM)
            optimizer.step()
    
evaluation = np.mean(evaluate(agent))
print('evaluation = ', np.mean(evaluation))
torch.save(agent, os.path.join(SAVE_PATH, 'ppo.final.torch'))


run id =  f3e84445
output folder: e:\ML\NLP\Reinforcement Learning\policy_gradient\ppo\atari\output\f3e84445
next obs =  torch.Size([3, 4, 84, 84])
next done =  torch.Size([3])


reward = 32.62, global_step = 6143229, best_score = 48.15, loss=-0.07: 100%|██████████| 1000/1000 [4:22:47<00:00, 15.77s/it] 


NameError: name 'evaluate' is not defined

# Evaluation

In [ ]:
next_obs, _ = envs.reset()
total_rewards = []
episodes = 10

while len(total_rewards) < episodes: 
    next_obs = torch.Tensor(next_obs)
    with torch.no_grad():
        action, log_prob, _, value = agent.get_action_and_value(next_obs)

    next_obs, reward, terminated, truncated, info = envs.step(action.numpy())

    if 'final_info' in info:
        for data in info['final_info']:
            if data:
                reward = data['episode']['r'][0]
                print(reward)
                total_rewards.append(reward)


268.0151
266.8834
260.6238
75.81549
267.05634
263.5786
266.1446
258.57123
263.439
-70.78032


In [ ]:
ENV_ARGS['render_mode'] = 'human'
envs1 = gym.vector.AsyncVectorEnv(
    [lambda : make_env(gamma= 0.99, **ENV_ARGS) for _ in range(NUM_ENVS)]
) 

next_obs, _ = envs1.reset()
total_rewards = []
episodes = 10

while len(total_rewards) < episodes: 
    next_obs = torch.Tensor(next_obs)
    with torch.no_grad():
        action, log_prob, _, value = agent.get_action_and_value(next_obs)

    next_obs, reward, terminated, truncated, info = envs1.step(action.numpy())

    if 'final_info' in info:
        for data in info['final_info']:
            if data:
                reward = data['episode']['r'][0]
                print(reward)
                total_rewards.append(reward)
    
del ENV_ARGS['render_mode']
envs1.close()

Exception ignored in: <function AsyncVectorEnv.__del__ at 0x132e0e310>
Traceback (most recent call last):
  File "/Users/jamesnguyen/anaconda3/envs/torch/lib/python3.9/site-packages/gymnasium/vector/async_vector_env.py", line 546, in __del__
    self.close(terminate=True)
  File "/Users/jamesnguyen/anaconda3/envs/torch/lib/python3.9/site-packages/gymnasium/vector/vector_env.py", line 265, in close
    self.close_extras(**kwargs)
  File "/Users/jamesnguyen/anaconda3/envs/torch/lib/python3.9/site-packages/gymnasium/vector/async_vector_env.py", line 461, in close_extras
    function(timeout)
  File "/Users/jamesnguyen/anaconda3/envs/torch/lib/python3.9/site-packages/gymnasium/vector/async_vector_env.py", line 321, in step_wait
    obs, rew, terminated, truncated, info = result
TypeError: cannot unpack non-iterable NoneType object
/Users/jamesnguyen/anaconda3/envs/torch/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated al

25.542747
307.70154
304.33347
-23.915497
305.6151
304.5836
302.65884
303.71445
305.96878
303.6156


# Optuna optimization